Modelling of supply points for soft-sensor for NUWEE.

In [1]:
import os
import pandas as pd
import numpy as np

import optuna
import torch

import plotly.graph_objects as go

In [2]:
utils_folder = os.path.join("..", "..", "utils")

data_folder = os.path.join("..", "..", "data")
clean_data_folder = os.path.join(data_folder, "Clean Data")
metadata_folder = os.path.join(data_folder, "Metadata")
plot_folder = os.path.join(data_folder, "Plots")

sensor_folder = os.path.join(clean_data_folder, "sensors")

nuwee_figures_folder = os.path.join(plot_folder, "nuwee")

In [3]:
grab_df = pd.read_excel(os.path.join(clean_data_folder, "modelling_grab.xlsx"))

nuwee_site1_df = pd.read_excel(os.path.join(clean_data_folder, 'nuwee', 'Site1_tabular.xlsx'))
nuwee_site2_df = pd.read_excel(os.path.join(clean_data_folder, 'nuwee', 'Site2_tabular.xlsx'))
nuwee_site3_df = pd.read_excel(os.path.join(clean_data_folder, 'nuwee', 'Site3_tabular.xlsx'))

In [ ]:
grab_df

In [5]:
grab_df.dropna(inplace=True)

In [6]:
grab_df['DateTime'] = pd.to_datetime(grab_df['DateTime'])

In [ ]:
nuwee_site1_df['DateTime'] = pd.to_datetime(nuwee_site1_df['DateTime'])
nuwee_site2_df['DateTime'] = pd.to_datetime(nuwee_site2_df['DateTime'])
nuwee_site3_df['DateTime'] = pd.to_datetime(nuwee_site3_df['DateTime'])

In [ ]:
codes_dict = {}
for cluster in grab_df['Cluster'].unique():
    print(f'Cluster {cluster}')
    codes = grab_df[grab_df['Cluster'] == cluster]['Code'].unique().tolist()
    codes_dict[cluster] = codes
    print(codes)

In [ ]:
nuwee_site1_df.columns

In [10]:
common_columns = nuwee_site1_df.columns.difference(['DateTime', 'Sampling Point', 'TTHMs'])

In [11]:
grab_df = grab_df[['DateTime', 'Cluster', 'Code', 'TTHMs'] + common_columns.tolist()]

In [12]:
cluster_0_df = grab_df[grab_df['Cluster'] == 0].copy()
cluster_1_df = grab_df[grab_df['Cluster'] == 1].copy()
cluster_2_df = grab_df[grab_df['Cluster'] == 2].copy()

# NUWEE Data preprocessing

We are going to fill deal with missing values and imputation.

In [ ]:
nuwee_site1_df.isna().sum()

In [ ]:
nuwee_site2_df.isna().sum()


In [ ]:
nuwee_site3_df.isna().sum()

In [16]:
# First we nee to clean the nuwee data by removing the rows with all missing values, then impute the rest
nuwee_site1_df.dropna(how='all', subset=common_columns, inplace=True)
nuwee_site2_df.dropna(how='all', subset=common_columns, inplace=True)
nuwee_site3_df.dropna(how='all', subset=common_columns, inplace=True)

In [17]:
# then, we need to remove the rows with all missing values in the TTHMs column
nuwee_site1_df.dropna(how='all', subset=['TTHMs'], inplace=True)
nuwee_site2_df.dropna(how='all', subset=['TTHMs'], inplace=True)
nuwee_site3_df.dropna(how='all', subset=['TTHMs'], inplace=True)

In [18]:
nuwee_site1_df.reset_index(drop=True, inplace=True)
nuwee_site2_df.reset_index(drop=True, inplace=True)
nuwee_site3_df.reset_index(drop=True, inplace=True)

In [ ]:
nuwee_site1_df.isna().sum()

In [ ]:
nuwee_site2_df.isna().sum()

In [ ]:
nuwee_site3_df.isna().sum()

In [ ]:
# now we can impute the missing values in the common columns
import miceforest as mf

In [ ]:
# create a kernel for each site
kernel = mf.ImputationKernel(
    data=nuwee_site1_df[common_columns],
    variable_schema=common_columns.tolist(),
    random_state=42,
    mean_match_strategy='shap',
)

kernel.mice(5, verbose=True)

In [24]:
nuwee_site1_df[common_columns] = kernel.complete_data(dataset=0)

In [ ]:
# create a kernel for each site
kernel = mf.ImputationKernel(
    data=nuwee_site2_df[common_columns],
    variable_schema=common_columns.tolist(),
    random_state=42,
    mean_match_strategy='shap',
)

kernel.mice(5, verbose=True)

In [26]:
nuwee_site2_df[common_columns] = kernel.complete_data(dataset=0)

In [ ]:
# create a kernel for each site
kernel = mf.ImputationKernel(
    data=nuwee_site3_df[common_columns],
    variable_schema=common_columns.tolist(),
    random_state=42,
    mean_match_strategy='shap',
)

kernel.mice(5, verbose=True)

In [28]:
nuwee_site3_df[common_columns] = kernel.complete_data(dataset=0)

In [ ]:
# final check
nuwee_site1_df.isna().sum()

In [ ]:
nuwee_site2_df.isna().sum()

In [ ]:
nuwee_site3_df.isna().sum()

# Clustering based on Mahalanobis distance

In [32]:
# Compute mean and covariance for each cluster
cluster_0_mean = cluster_0_df[common_columns].mean()
cluster_0_cov = cluster_0_df[common_columns].cov()
cluster_1_mean = cluster_1_df[common_columns].mean()
cluster_1_cov = cluster_1_df[common_columns].cov()
cluster_2_mean = cluster_2_df[common_columns].mean()
cluster_2_cov = cluster_2_df[common_columns].cov()

In [33]:
clusters_stats = {
    0: {
        'mean': cluster_0_mean,
        'cov': cluster_0_cov
    },
    1: {
        'mean': cluster_1_mean,
        'cov': cluster_1_cov
    },
    2: {
        'mean': cluster_2_mean,
        'cov': cluster_2_cov
    }
}

In [34]:
from scipy.spatial.distance import mahalanobis

def assign_cluster(row, clusters_stats):
    distances = {}
    for cluster, stats in clusters_stats.items():
        mean = stats['mean']
        cov = stats['cov']
        inv_cov = np.linalg.inv(cov)
        distance = mahalanobis(row[common_columns], mean, inv_cov)
        distances[cluster] = distance
    return min(distances, key=distances.get)

In [35]:
# nuwee site 1

nuwee_site1_df['Cluster'] = -1
for i, row in nuwee_site1_df.iterrows():
    cluster = assign_cluster(row, clusters_stats)
    nuwee_site1_df.at[i, 'Cluster'] = cluster

In [ ]:
nuwee_site1_df['Cluster'].value_counts()

In [37]:
# nuwee site 2
nuwee_site2_df['Cluster'] = -1

for i, row in nuwee_site2_df.iterrows():
    cluster = assign_cluster(row, clusters_stats)
    nuwee_site2_df.at[i, 'Cluster'] = cluster

In [ ]:
nuwee_site2_df['Cluster'].value_counts()

In [39]:
# nuwee site 3
nuwee_site3_df['Cluster'] = -1

for i, row in nuwee_site3_df.iterrows():
    cluster = assign_cluster(row, clusters_stats)
    nuwee_site3_df.at[i, 'Cluster'] = cluster

In [ ]:
nuwee_site3_df['Cluster'].value_counts()

# PCA Cluster Visualization

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)

cluster_0_pca = pca.fit_transform(cluster_0_df[common_columns])
ev_cluster_0 = pca.explained_variance_ratio_

nuwee_site1_pca = pca.fit_transform(nuwee_site1_df[common_columns])
ev_nuwee_site1 = pca.explained_variance_ratio_
nuwee_site2_pca = pca.fit_transform(nuwee_site2_df[common_columns])
ev_nuwee_site2 = pca.explained_variance_ratio_
nuwee_site3_pca = pca.fit_transform(nuwee_site3_df[common_columns])
ev_nuwee_site3 = pca.explained_variance_ratio_

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=cluster_0_pca[:, 0],
        y=cluster_0_pca[:, 1],
        mode='markers',
        marker=dict(
            color='blue'
        ),
        name=f'Milan Custer 0 (Expl. Var: {ev_cluster_0[0]:.2%})'
    )
)

fig.add_trace(
    go.Scatter(
        x=nuwee_site1_pca[:, 0],
        y=nuwee_site1_pca[:, 1],
        mode='markers',
        marker=dict(
            color='green'
        ),
        name=f'Nuwee Site 1 (Expl. Var: {ev_nuwee_site1[0]:.2%})'
    )
)

fig.add_trace(
    go.Scatter(
        x=nuwee_site2_pca[:, 0],
        y=nuwee_site2_pca[:, 1],
        mode='markers',
        marker=dict(
            color='orange'
        ),
        name=f'Nuwee Site 2 (Expl. Var: {ev_nuwee_site2[0]:.2%})'
    )
)

fig.add_trace(
    go.Scatter(
        x=nuwee_site3_pca[:, 0],
        y=nuwee_site3_pca[:, 1],
        mode='markers',
        marker=dict(
            color='purple'
        ),
        name=f'Nuwee Site 3 (Expl. Var: {ev_nuwee_site3[0]:.2%})'
    )
)

fig.update_layout(
    title='PCA Visualization of Clusters Across Sites',
    xaxis_title='Principal Component 1',
    yaxis_title='Principal Component 2',
    legend_title='Sites',
    legend=dict(
        x=0.65,
        y=1.0,
        xanchor='left',
        yanchor='top',
        bgcolor='rgba(255, 255, 255, 0.7)',
    ),
    font=dict(
        family="Arial, sans-serif",
        size=18,
        color="black"
    ),
    margin=dict(
        l=30,
        r=10,
        b=20,
        t=40,
        pad=4
    ),
    width=1000,
    height=800
)

fig.write_image(os.path.join(nuwee_figures_folder, "pca.png"), scale=3)

fig.show()

# Modelling

In [42]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [43]:
# add the month as a feature to give time context
cluster_0_df['Month'] = cluster_0_df["DateTime"].dt.month

In [44]:
nuwee_site1_df['Month'] = nuwee_site1_df['DateTime'].dt.month
nuwee_site2_df['Month'] = nuwee_site2_df['DateTime'].dt.month
nuwee_site3_df['Month'] = nuwee_site3_df['DateTime'].dt.month

In [ ]:
common_columns.append(pd.Index(['Month']))

In [46]:
# Prepare the data
# All the data from cluster 0 will be used for training

cluster_0_df.set_index('DateTime', inplace=True)
X, y = cluster_0_df[common_columns], cluster_0_df['TTHMs']

# scale the data
scaler = MinMaxScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [ ]:
X_train.shape, X_test.shape

## PLS

The Partial Least Squares regression (PLS) is a method which reduces the variables, used to predict, to a smaller set of predictors. These predictors are then used to perform a regression.

It projects the predictors (independent variables) and the response variable (dependent variable) into a new space that maximizes the covariance between them. The procedure identifies components (latent variables) that explain the most variance in the predictors while also being predictive of the response variable.

In [48]:
from sklearn.cross_decomposition import PLSRegression

In [49]:
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_squared_error


In [50]:
def fit_and_validate_pls_model(
    X,
    y,
    train_index,
    val_index,
    params,
):
    X_tr, X_val = X.iloc[train_index], X.iloc[val_index]
    y_tr, y_val = y.iloc[train_index], y.iloc[val_index]
    
    
    n_components = params["n_components"]
    tol = params["tol"]

    model = PLSRegression(
        n_components=n_components,
        tol=tol,
        scale=False,
        max_iter=1000,
    )
    
    model.fit(X_tr, y_tr)
    
    y_val_pred = model.predict(X_val)

    # return metrics
    return np.sqrt(mean_squared_error(y_val, y_val_pred))

In [51]:
def objective(trial: optuna.trial.Trial, X_cv, y_cv) -> float:
    
    config= {
        
        "n_components": trial.suggest_int("n_components", 2, X_cv.shape[1]),
        "tol": trial.suggest_float("tol", 1e-6, 1e-1),
        
    }
    cv = LeaveOneOut()
    cv_rmse = np.zeros((cv.get_n_splits(X_cv)))
    for i, (train_index, test_index) in enumerate(
        cv.split(X_cv, y_cv)
    ):
        cv_rmse[i] = fit_and_validate_pls_model(
            X_cv,
            y_cv,
            train_index,
            test_index,
            config,
        )
        
    # saving the individual fold holdout metrics
    # uncomment this line if you don't want this
    # trial.set_user_attr("split_rmse", cv_rmse)
    
    return np.mean(cv_rmse)

In [52]:
if os.path.exists(f"nuwee_sqlites/PLS.sqlite3"):
    
    study = optuna.load_study(
        study_name=f"Hyperparameter Tuning - PLS",
        storage=f"sqlite:///nuwee_sqlites/PLS.sqlite3",
    )

else:
    
    study = optuna.create_study(
        study_name=f"Hyperparameter Tuning - PLS",
        storage=f"sqlite:///nuwee_sqlites/PLS.sqlite3",
        direction="minimize",
        load_if_exists=True,
    )

    study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=100, show_progress_bar=True)

pls_study = study

## SVR

In [53]:
# import SVR from sklearn
from sklearn.svm import SVR

In [54]:
# print the parameters of the model
svr = SVR()

In [55]:
kernel = [
    "linear",
    "rbf",
    "sigmoid",
]

In [56]:
def fit_and_validate_svr_model(
    X,
    y,
    train_index,
    val_index,
    params,
):
    X_tr, X_val = X.iloc[train_index], X.iloc[val_index]
    y_tr, y_val = y.iloc[train_index], y.iloc[val_index]
    
    kernel = params["kernel"]
    C = params["C"]
    epsilon = params["epsilon"]
    gamma = params["gamma"]

    model = SVR(
        kernel=kernel,
        C=C,
        epsilon=epsilon,
        gamma=gamma,
    )
    
    model.fit(X_tr, y_tr)
    
    y_val_pred = model.predict(X_val)

    # return metrics
    return np.sqrt(mean_squared_error(y_val, y_val_pred))

In [57]:
def objective(trial: optuna.trial.Trial, X_cv, y_cv) -> float:
    
    config= {
        
        "kernel": trial.suggest_categorical("kernel", kernel),
        "C": trial.suggest_float("C", 1e-6, 1, log=True),
        "epsilon": trial.suggest_float("epsilon", 1e-6, 1, log=True),
        "gamma": trial.suggest_float("gamma", 1e-6, 1, log=True),
        
    }
    cv = LeaveOneOut()
    cv_rmse = np.zeros((cv.get_n_splits(X_cv)))
    for i, (train_index, test_index) in enumerate(
        cv.split(X_cv, y_cv)
    ):
        cv_rmse[i] = fit_and_validate_svr_model(
            X_cv,
            y_cv,
            train_index,
            test_index,
            config,
        )
        
    # saving the individual fold holdout metrics
    # uncomment this line if you don't want this
    # trial.set_user_attr("split_rmse", cv_rmse)
    
    return np.mean(cv_rmse)

In [58]:
if os.path.exists(f"nuwee_sqlites/SVR.sqlite3"):
    
    study = optuna.load_study(
        study_name=f"Hyperparameter Tuning - SVR",
        storage=f"sqlite:///nuwee_sqlites/SVR.sqlite3",
    )

else:
    
    study = optuna.create_study(
        study_name=f"Hyperparameter Tuning - SVR",
        storage=f"sqlite:///nuwee_sqlites/SVR.sqlite3",
        direction="minimize",
        load_if_exists=True,
    )

    study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=100, show_progress_bar=True)

svr_study = study

## QRNN

In [59]:
from quantnn.qrnn import QRNN

In [60]:
quantiles = np.linspace(0.01, 0.99, 99)

def fit_and_validate_qrnn_model(
    X,
    y,
    train_index,
    val_index,
    params,
):
    X_tr, X_val = X.iloc[train_index].to_numpy(), X.iloc[val_index].to_numpy()
    y_tr, y_val = y.iloc[train_index], y.iloc[val_index]
    
    
    n_layers = params["n_layers"]
    n_units = params["n_units"]
    activation = params["activation"]

    model = QRNN(
        n_inputs=X_tr.shape[1],
        quantiles=quantiles,
        model=(n_layers, n_units, activation),
    )
    
    n_epochs = 50
    optimizer = torch.optim.AdamW(model.model.parameters())
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs)
    
    model.train(
        training_data=(np.array(X_tr), np.array(y_tr)),
        optimizer=optimizer,
        scheduler=scheduler,
        n_epochs=n_epochs,
        device="cpu",
        batch_size=params["batch_size"],
        logger=None,
        
    )
    
    with torch.no_grad():
        y_val_pred = model.predict(X_val)
    

    # return metrics
    return np.sqrt(mean_squared_error(y_val, y_val_pred.mean(axis=-1)))

In [61]:
activations = [
    "elu",
    "hardshrink",
    "hardtanh",
    "prelu",
    "relu",
    "selu",
    "celu",
    "sigmoid",
    "softplus",
    "softmin",
]

In [62]:
def objective(trial: optuna.trial.Trial, X_cv, y_cv) -> float:
    
    config= {
        
        "n_layers": trial.suggest_int("n_layers", 1, 3),
        "n_units": trial.suggest_int("n_units", 32, 512, log=True),
        "activation": trial.suggest_categorical("activation", activations),
        "batch_size": trial.suggest_categorical("batch_size", [4, 8, 16]),
    }

    cv = LeaveOneOut()
    cv_rmse = np.zeros((cv.get_n_splits(X_cv)))
    for i, (train_index, test_index) in enumerate(
        cv.split(X_cv, y_cv)
    ):
        cv_rmse[i] = fit_and_validate_qrnn_model(
            X_cv,
            y_cv,
            train_index,
            test_index,
            config,
        )
        
    # saving the individual fold holdout metrics
    # uncomment this line if you don't want this
    # trial.set_user_attr("split_rmse", cv_rmse)
    
    return np.mean(cv_rmse)

In [63]:
if os.path.exists(f"nuwee_sqlites/QRNN.sqlite3"):
    
    study = optuna.load_study(
        study_name=f"Hyperparameter Tuning - QRNN",
        storage=f"sqlite:///nuwee_sqlites/QRNN.sqlite3",
    )

else:
    
    study = optuna.create_study(
        study_name=f"Hyperparameter Tuning - QRNN",
        storage=f"sqlite:///nuwee_sqlites/QRNN.sqlite3",
        direction="minimize",
        load_if_exists=True,
    )

    study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=100, show_progress_bar=True)

qrnn_study = study

## XGBoost

In [64]:
from xgboost import XGBRegressor

In [65]:
def fit_and_validate_xgb_model(
    X,
    y,
    train_index,
    val_index,
    params,
):
    X_tr, X_val = X.iloc[train_index, :], X.iloc[val_index, :]
    y_tr, y_val = y.iloc[train_index], y.iloc[val_index]

    model = XGBRegressor(random_state=42, **params)

    # train model
    _ = model.fit(X_tr, y_tr)

    # obtain predictions
    y_val_pred = model.predict(X_val)

    # return metrics
    return np.sqrt(mean_squared_error(y_val, y_val_pred))

In [66]:
def objective(trial: optuna.trial.Trial, X_cv, y_cv) -> float:
    eta = trial.suggest_float("eta", 1e-5, 1, log=True)
    reg_lambda = trial.suggest_float("reg_lambda", 1e-8, 1, log=True)
    reg_alpha = trial.suggest_float("reg_alpha", 1e-8, 1, log=True)
    learning_rate = trial.suggest_float(
        "learning_rate", 1e-5, 1, log=True
    )
    n_estimators = trial.suggest_int("n_estimators", 1, 500)
    updater = trial.suggest_categorical(
        "updater", ["shotgun", "coord_descent"]
    )

    params = {
        "objective": "reg:squarederror",
        "booster": "gblinear",
        "eta": eta,
        "reg_lambda": reg_lambda,
        "reg_alpha": reg_alpha,
        "learning_rate": learning_rate,
        "updater": updater,
        "n_estimators": n_estimators,
        "eval_metric": "rmse",
    }

    cv = LeaveOneOut()
    cv_rmse = np.zeros((cv.get_n_splits(X_cv)))
    for i, (train_index, test_index) in enumerate(
        cv.split(X_cv, y_cv)
    ):
        cv_rmse[i] = fit_and_validate_xgb_model(
            X_cv,
            y_cv,
            train_index,
            test_index,
            params,
        )

    # saving the individual fold holdout metrics
    # uncomment this line if you don't want this
    # trial.set_user_attr("split_rmse", cv_rmse)

    return np.mean(cv_rmse)

In [67]:
if os.path.exists(f"nuwee_sqlites/XGB.sqlite3"):
    
    study = optuna.load_study(
        study_name=f"Hyperparameter Tuning - XGB",
        storage=f"sqlite:///nuwee_sqlites/XGB.sqlite3",
    )

else:
    
    study = optuna.create_study(
        study_name=f"Hyperparameter Tuning - XGB",
        storage=f"sqlite:///nuwee_sqlites/XGB.sqlite3",
        direction="minimize",
        load_if_exists=True,
    )

    study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=100, show_progress_bar=True)

xgb_study = study

# Comparison

In [68]:
# get all the studies

best_studies= {
    "PLS": pls_study.best_trial,
    "SVR": svr_study.best_trial,
    "QRNN": qrnn_study.best_trial,
    "XGB": xgb_study.best_trial,
}

In [69]:
comparison_df = pd.DataFrame(
    columns=['RMSE'],
    index=list(best_studies.keys()),
)

for model, study in best_studies.items():
    comparison_df.loc[model, :] = np.round(study.value, 3)
    

In [ ]:
comparison_df

# Model Prediction with all common features

Since all the points were associated to cluster 0, we are going to use the model that performed best on all the features for cluster 0 and we are going to use it on these samples.

In [71]:
if os.path.exists(f"nuwee_sqlites/XGB.sqlite3"):
    
    xgb_study = optuna.load_study(
        study_name=f"Hyperparameter Tuning - XGB",
        storage=f"sqlite:///nuwee_sqlites/XGB.sqlite3",
    )

else:
    
    raise FileNotFoundError(
        f"SQLite file not found. Please check the path."
    )

In [72]:
from sklearn.metrics import mean_absolute_percentage_error

In [73]:
# scale the sites df
nuwee_site1_df[common_columns] = scaler.transform(nuwee_site1_df[common_columns])
nuwee_site2_df[common_columns] = scaler.transform(nuwee_site2_df[common_columns])
nuwee_site3_df[common_columns] = scaler.transform(nuwee_site3_df[common_columns])

In [74]:
n_iterations = 50

milan_preds = []
site1_preds = []
site2_preds = []
site3_preds = []

for _ in range(n_iterations):

    xgb_best_trial = xgb_study.best_trial
    
    model = XGBRegressor(
        random_state=42,
        objective="reg:squarederror",
        booster="gblinear",
        eta=xgb_best_trial.params['eta'],
        reg_lambda=xgb_best_trial.params['reg_lambda'],
        reg_alpha=xgb_best_trial.params['reg_alpha'],
        learning_rate=xgb_best_trial.params['learning_rate'],
        updater=xgb_best_trial.params['updater'],
        n_estimators=xgb_best_trial.params['n_estimators'],
        eval_metric="rmse"
    )
    
    model.fit(X_train, y_train)
    
    milan_preds.append(model.predict(X_test))
    site1_preds.append(model.predict(nuwee_site1_df[common_columns]))
    site2_preds.append(model.predict(nuwee_site2_df[common_columns]))
    site3_preds.append(model.predict(nuwee_site3_df[common_columns]))

eval_preds = {
    "y_test_milan": y_test,
    "y_test1": nuwee_site1_df['TTHMs'],
    "y_test2": nuwee_site2_df['TTHMs'],
    "y_test3": nuwee_site3_df['TTHMs'],
    "y_test_mean_milan": np.mean(milan_preds, axis=0),
    "y_test_mean1": np.mean(site1_preds, axis=0),
    "y_test_mean2": np.mean(site2_preds, axis=0),
    "y_test_mean3": np.mean(site3_preds, axis=0),
    "y_test_lower_milan": np.quantile(milan_preds, 0.025, axis=0),
    "y_test_lower1": np.quantile(site1_preds, 0.025, axis=0),
    "y_test_lower2": np.quantile(site2_preds, 0.025, axis=0),
    "y_test_lower3": np.quantile(site3_preds, 0.025, axis=0),
    "y_test_upper_milan": np.quantile(milan_preds, 0.975, axis=0),
    "y_test_upper1": np.quantile(site1_preds, 0.975, axis=0),
    "y_test_upper2": np.quantile(site2_preds, 0.975, axis=0),
    "y_test_upper3": np.quantile(site3_preds, 0.975, axis=0),
    "rmse_milan": np.sqrt(mean_squared_error(y_test.values, np.mean(milan_preds, axis=0))),
    "rmse1": np.sqrt(mean_squared_error(nuwee_site1_df['TTHMs'].values, np.mean(site1_preds, axis=0))),
    "rmse2": np.sqrt(mean_squared_error(nuwee_site2_df['TTHMs'].values, np.mean(site2_preds, axis=0))),
    "rmse3": np.sqrt(mean_squared_error(nuwee_site3_df['TTHMs'].values, np.mean(site3_preds, axis=0))),
    "mape_milan": mean_absolute_percentage_error(y_test.values, np.mean(milan_preds, axis=0)),
    "mape1": mean_absolute_percentage_error(nuwee_site1_df['TTHMs'].values, np.mean(site1_preds, axis=0)),
    "mape2": mean_absolute_percentage_error(nuwee_site2_df['TTHMs'].values, np.mean(site2_preds, axis=0)),
    "mape3": mean_absolute_percentage_error(nuwee_site3_df['TTHMs'].values, np.mean(site3_preds, axis=0))
}

In [ ]:
print("Milan RMSE: " + str(eval_preds['rmse_milan']))
print("Site 1 RMSE: " + str(eval_preds["rmse1"]))
print("Site 2 RMSE: " + str(eval_preds["rmse2"]))
print("Site 3 RMSE: " + str(eval_preds["rmse3"]))
print()
print("Milan MAPE: " + str(eval_preds["mape_milan"]))
print("Site 1 MAPE: " + str(eval_preds["mape1"]))
print("Site 2 MAPE: " + str(eval_preds["mape2"]))
print("Site 3 MAPE: " + str(eval_preds["mape3"]))


In [76]:
import matplotlib.pyplot as plt

In [77]:
# increase the font size
plt.rcParams.update({'font.size': 16})

In [ ]:
# Site 1
y_test1 = eval_preds["y_test1"]
y_test_mean1 = eval_preds["y_test_mean1"]

plt.figure(figsize=(10, 5))

# Get unique sampling points
unique_sampling_points = nuwee_site1_df['Sampling Point'].unique()

# Plot each sampling point with a different color
for sampling_point in unique_sampling_points:
    mask = nuwee_site1_df['Sampling Point'] == sampling_point
    plt.scatter(
        y_test1[mask], 
        y_test_mean1[mask], 
        label=sampling_point
    )

plt.plot([0, 9], [0, 9], "--", color="black")
plt.xlabel("True")
plt.ylabel("Predicted")
plt.legend(title="Sampling Point", fontsize=12)

plt.title("Site 1 - True vs Predicted")
plt.tight_layout()
plt.savefig(os.path.join(nuwee_figures_folder, "site1_true_vs_predicted.png"))
plt.show()

In [ ]:
# Site 2
y_test2 = eval_preds["y_test2"]
y_test_mean2 = eval_preds["y_test_mean2"]

# Get unique sampling points
unique_sampling_points = nuwee_site2_df['Sampling Point'].unique()

# Plot each sampling point with a different color
plt.figure(figsize=(10, 5))
for sampling_point in unique_sampling_points:
    mask = nuwee_site2_df['Sampling Point'] == sampling_point
    plt.scatter(
        y_test2[mask], 
        y_test_mean2[mask], 
        label=sampling_point
    )

plt.plot([0, 14], [0, 14], "--", color="black")
plt.xlabel("True")
plt.ylabel("Predicted")
plt.legend(title="Sampling Point", fontsize=12)
plt.title("Site 2 - True vs Predicted (by Sampling Point)")
plt.tight_layout()
plt.savefig(os.path.join(nuwee_figures_folder, "site2_true_vs_predicted.png"))
plt.show()

In [ ]:
# Site 3
y_test3 = eval_preds["y_test3"]
y_test_mean3 = eval_preds["y_test_mean3"]

# Get unique sampling points
unique_sampling_points = nuwee_site3_df['Sampling Point'].unique()

# Plot each sampling point with a different color
plt.figure(figsize=(10, 5))
for sampling_point in unique_sampling_points:
    mask = nuwee_site3_df['Sampling Point'] == sampling_point
    plt.scatter(
        y_test3[mask], 
        y_test_mean3[mask], 
        label=sampling_point
    )

plt.plot([0, 14], [0, 14], "--", color="black")
plt.xlabel("True")
plt.ylabel("Predicted")
plt.legend(title="Sampling Point", fontsize=12)
plt.title("Site 3 - True vs Predicted (by Sampling Point)")
plt.tight_layout()
plt.savefig(os.path.join(nuwee_figures_folder, "site3_true_vs_predicted.png"))
plt.show()

In [81]:
import plotly.graph_objects as go

In [ ]:
# Site 1
y_test = eval_preds["y_test1"]
y_pred_mean = eval_preds["y_test_mean1"]
y_pred_lower = eval_preds["y_test_lower1"]
y_pred_upper = eval_preds["y_test_upper1"]
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=nuwee_site1_df['DateTime'],
        y=nuwee_site1_df['TTHMs'],
        mode="markers",
        name="True TTHMs",
        line=dict(color="black"),
        marker=dict(size=10),
    )
)

fig.add_trace(
    go.Scatter(
        x=nuwee_site1_df['DateTime'],
        y=y_pred_mean,
        mode="markers",
        name="Predicted TTHMs (95% PI)",
        line=dict(color="green"),
        marker=dict(size=10),
        error_y=dict(
            type='data',
            symmetric=False,
            array=y_pred_upper,
            arrayminus=y_pred_lower,
            thickness=2,
            width=5,
            color="green",
        ),
    )
)


# get the legend inside the plot
fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1,
        xanchor="right",
        x=1,
    ),
    margin=dict(l=10, r=10, t=30, b=10),
    title="Site 1"
)

fig.update_xaxes(title_text="Time")
fig.update_yaxes(title_text="TTHMs (µg/L)")

# fig.update_yaxes(range=[0, 25])

# update the overall font
fig.update_layout(font=dict(family="Arial", size=18))

fig.write_image(os.path.join(nuwee_figures_folder, "site1_pred_with_CI.png"), scale=3)

fig.show()

In [ ]:
# Site 2
y_test = eval_preds["y_test2"]
y_pred_mean = eval_preds["y_test_mean2"]
y_pred_lower = eval_preds["y_test_lower2"]
y_pred_upper = eval_preds["y_test_upper2"]
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=nuwee_site2_df['DateTime'],
        y=nuwee_site2_df['TTHMs'],
        mode="markers",
        name="True TTHMs",
        line=dict(color="black"),
        marker=dict(size=10),
    )
)

fig.add_trace(
    go.Scatter(
        x=nuwee_site2_df['DateTime'],
        y=y_pred_mean,
        mode="markers",
        name="Predicted TTHMs (95% PI)",
        line=dict(color="green"),
        marker=dict(size=10),
        error_y=dict(
            type='data',
            symmetric=False,
            array=y_pred_upper,
            arrayminus=y_pred_lower,
            thickness=2,
            width=5,
            color="green",
        ),
    )
)

# get the legend inside the plot
fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1,
        xanchor="right",
        x=1,
    ),
    margin=dict(l=10, r=10, t=30, b=10),
    title="Site 2"
)

fig.update_xaxes(title_text="Time")
fig.update_yaxes(title_text="TTHMs (µg/L)")

# fig.update_yaxes(range=[0, 25])

# update the overall font
fig.update_layout(font=dict(family="Arial", size=18))

fig.write_image(os.path.join(nuwee_figures_folder, "site2_pred_with_CI.png"), scale=3)

fig.show()

In [ ]:
# Site 3
y_test3 = eval_preds["y_test3"]
y_test_mean3 = eval_preds["y_test_mean3"]
y_test_lower3 = eval_preds["y_test_lower3"]
y_test_upper3 = eval_preds["y_test_upper3"]

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=nuwee_site3_df['DateTime'],
        y=nuwee_site3_df['TTHMs'],
        mode="markers",
        name="True TTHMs",
        line=dict(color="black"),
        marker=dict(size=10),
    )
)

fig.add_trace(
    go.Scatter(
        x=nuwee_site3_df['DateTime'],
        y=y_test_mean3,
        mode="markers",
        name="Predicted TTHMs (95% PI)",
        line=dict(color="green"),
        marker=dict(size=10),
        error_y=dict(
            type='data',
            symmetric=False,
            array=y_test_upper3,
            arrayminus=y_test_lower3,
            thickness=2,
            width=5,
            color="green",
        ),
    )
)

fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1,
        xanchor="right",
        x=1,
    ),
    margin=dict(l=10, r=10, t=30, b=10),
    title="Site 3"
)

fig.update_xaxes(title_text="Time")
fig.update_yaxes(title_text="TTHMs (µg/L)")

# fig.update_yaxes(range=[0, 25])

# update the overall font
fig.update_layout(font=dict(family="Arial", size=18))

fig.write_image(os.path.join(nuwee_figures_folder, "site3_pred_with_CI.png"), scale=3)

fig.show()

# Try Cluster 0 based on Feature Selection

In the feature_selection/supply_points.ipynb notebook, we assessed that there can be improvements if we perform feature selection. For cluster 0, the best results were achieved with XGBoost with features: ['Conductivity (uS/cm)', 'TOC (mg/L)', 'Temperature (°C)'].
Let's try to use that model with those 3 features.

In [85]:
from tqdm.notebook import tqdm_notebook

In [86]:
feature_selection_xgb_folder = os.path.join("..", "feature_selection", "supply_points_sqlites", "xgboost")

In [87]:
# reload grab_df and call it df
df = pd.read_excel(os.path.join(clean_data_folder, "modelling_grab.xlsx"))

In [ ]:
df.columns

In [89]:
features = df.columns.difference(["DateTime", "Code", "TTHMs", "Cluster"], sort=False).to_list()

In [90]:
from itertools import combinations

cluster_name = "cluster_0"

feature_combinations = []
for i in range(1, len(features) + 1):
    feature_combinations.extend(combinations(features, i))

In [91]:
feature_combinations = [list(comb) for comb in feature_combinations]

In [ ]:
# get the configuration that performed the best
xgb_studies = {}

    
xgb_studies = {}
for feature_combination in tqdm_notebook(feature_combinations):
    
    study_name = f"Hyperparameter Tuning - XGB_{cluster_name}" + str(feature_combination)
    storage_path = f"sqlite:///{feature_selection_xgb_folder}/{cluster_name}" + str(feature_combination).replace('/', '_') + ".sqlite3"

    if os.path.exists(f"{feature_selection_xgb_folder}/{cluster_name}" + str(feature_combination).replace('/', '_') + ".sqlite3"):
        
        
        
        study = optuna.load_study(
            study_name=study_name,
            storage=storage_path,
        )
    
    else:
        
        raise FileExistsError(f"Study with name {study_name} at path {storage_path} does not exists.")
    
    xgb_studies[str(feature_combination)] = study

In [93]:
best_study_info = sorted(xgb_studies.items(), key=lambda x: x[1].best_value)[0]

In [ ]:
best_study_info

In [95]:
best_study_features = ['Conductivity (uS/cm)', 'TOC (mg/L)', 'Temperature (°C)']
best_study = best_study_info[-1]

In [96]:
df = df[['DateTime', 'Cluster', 'Code', 'TTHMs'] + best_study_features]
cl_0_df = df[df['Cluster'] == 0].copy()

In [97]:
cl_0_df.set_index('DateTime', inplace=True)
X_fs, y_fs = cl_0_df[best_study_features], cl_0_df['TTHMs']

# scale the data
scaler = MinMaxScaler()
X_fs = pd.DataFrame(scaler.fit_transform(X_fs), columns=X_fs.columns, index=X_fs.index)

X_train_fs, X_test_fs, y_train_fs, y_test_fs = train_test_split(X_fs, y_fs, test_size=0.2, shuffle=False)

In [98]:
from xgboost import XGBRegressor

In [99]:
n_iterations = 50

medians = []
lower = []
upper = []

for _ in range(n_iterations):
    
    model = XGBRegressor(
        random_state=42,
        objective="reg:squarederror",
        booster="gblinear",
        eta=best_study.best_trial.params["eta"],
        reg_lambda=best_study.best_trial.params["reg_lambda"],
        reg_alpha=best_study.best_trial.params["reg_alpha"],
        learning_rate=best_study.best_trial.params["learning_rate"],
        updater=best_study.best_trial.params["updater"],
        n_estimators=best_study.best_trial.params["n_estimators"],
    )
    
    # train model
    _ = model.fit(X_train_fs, y_train_fs)
    
    # obtain predictions
    y_test_median = model.predict(X_test_fs)
    y_test_lower = y_test_median - 1.96 * np.std(y_test_median)
    y_test_upper = y_test_median + 1.96 * np.std(y_test_median)

    
    medians.append(y_test_median)
    lower.append(y_test_lower)
    upper.append(y_test_upper)

eval = {
    "y_test": y_test_fs,
    "y_test_median": np.mean(medians, axis=0),
    "y_test_lower": np.mean(lower, axis=0),
    "y_test_upper": np.mean(upper, axis=0),
}


In [100]:
rmse = np.sqrt(mean_squared_error(y_test_fs.values, eval["y_test_median"]))

In [ ]:
rmse

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(y_test_fs, eval["y_test_median"], "o")
plt.plot([0, 14], [0, 14], "--")
plt.xlabel("True")
plt.ylabel("Predicted")

plt.title(f"{cluster_name} - True vs Predicted")
plt.show()

In [ ]:

y_test_fs = eval["y_test"]
y_pred_xgb_median = eval["y_test_median"]
y_pred_xgb_lower = eval["y_test_lower"]
y_pred_xgb_upper = eval["y_test_upper"]

y_test_fs.index = pd.to_datetime(y_test_fs.index)

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=y_test_fs.index,
        y=y_test_fs,
        mode="markers",
        name="True TTHMs",
        line=dict(color="black"),
        marker=dict(size=10),
    )
)

fig.add_trace(
    go.Scatter(
        x=y_test_fs.index,
        y=y_pred_xgb_median,
        mode="markers",
        name="Predicted TTHMs (95% PI)",
        line=dict(color="green"),
        marker=dict(size=10),
        error_y=dict(
            type='data',
            symmetric=False,
            array=y_pred_xgb_upper,
            arrayminus=y_pred_xgb_lower,
            thickness=2,
            width=5,
            color="green",
        ),
    )
)

# get the legend inside the plot
fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1,
        xanchor="right",
        x=1,
    ),
    margin=dict(l=10, r=10, t=30, b=10),
)

fig.update_xaxes(title_text="Time")
fig.update_yaxes(title_text="TTHMs (µg/L)")

fig.update_yaxes(range=[0, 25])

# update the overall font
fig.update_layout(font=dict(family="Arial", size=18))

# fig.write_image(
#     f"{cluster_name}.png",
#     scale=3,
# )

fig.show()